In [1]:
import numpy as np
import pandas as pd

In [50]:
hexin = pd.read_excel('../data/核心.xls')
renhang = pd.read_excel('../data/人行.xls')
yanyin = pd.read_excel('../data/验印.xls')

### 数据筛选

#### 条件筛选

In [4]:
# 核心数据筛选
hexin = hexin[(hexin['账户属性'] == '4 - 基本户') |
              (hexin['账户属性'] == '3 - 一般户') |
              (hexin['账户属性'] == '6 - 专用户（核准类）') |
              (hexin['账户属性'] == '23 - 专用户（备案类）') |
              (hexin['账户属性'] == '7 - 验资户')]

In [5]:
hexin.rename(columns={'账号/卡号':'账号'},inplace=True)

In [6]:
# 验印数据数据筛选
yanyin = yanyin[(yanyin['账户属性'] == '基本户') |
                (yanyin['账户属性'] == '一般户') |
                (yanyin['账户属性'] == '专用户（核准类）') |
                (yanyin['账户属性'] == '专用户（备案类）') |
                (yanyin['账户属性'] == '验资户')]

yanyin = yanyin[yanyin['销户标志'] == '正常']

yanyin = yanyin[yanyin['主从账户标志'] != '从账户']

#### 重复值处理

In [37]:
with open("../data/机构参数.txt", "r",encoding='utf-8') as f:  # 打开文件
    data = f.readlines()  # 读取文件
    print(data)

['机构号：419901\n', '机构名称：中原银行郑州商务外环路支行']


In [38]:
para=[]
with open("../data/机构参数.txt", "r",encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip('\n')
        para.append(line)
    

In [55]:
yanyin['销户日期']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
5246    NaN
5247    NaN
5248    NaN
5249    NaN
5250    NaN
Name: 销户日期, Length: 5251, dtype: object

In [52]:
renhang['销户日期'].fillna('2099-12-31', inplace=True)
renhang.sort_values(by=['开户日期', '销户日期'], ascending=False, inplace=True)

In [53]:
renhang['销户日期']

4905    2099-12-31
4906    2099-12-31
4901    2099-12-31
4902    2099-12-31
4903    2099-12-31
           ...    
8       2018-06-21
0       2016-07-21
10      2015-05-21
9       2015-03-27
11      2099-12-31
Name: 销户日期, Length: 4907, dtype: object

In [45]:
renhang[['开户日期', '销户日期']]

,开户日期,销户日期
4905,2020-09-30,2056
4906,2020-09-30,2056
4901,2020-09-29,2056
4902,2020-09-29,2056
4903,2020-09-29,2056
...,...,...
8,2014-12-26,2018-06-21
0,2014-12-26,2016-07-21
10,2014-12-26,2015-05-21
9,2014-12-26,2015-03-27


In [43]:
para[0].split('：')[1]

'419901'

In [26]:
hexin.sort_values(by=['开户日期','销户日期'], ascending=False, inplace=True)

In [25]:
hexin['销户日期'].fillna(20991231,inplace=True)

In [13]:
hexin.sort_values(by=['销户日期'], ascending=True, inplace=True)

In [27]:
hexin.to_excel('ll.xlsx',index=False)

In [24]:
hexin.sort_values(by=['开户日期'], ascending=False, inplace=True)
hexin.drop_duplicates(subset=['账号'], keep='first', inplace=True)
hexin = hexin.sort_values(by=['账号']).reset_index(drop=True)

In [25]:
renhang.sort_values(by=['开户日期'], ascending=False, inplace=True)
renhang.drop_duplicates(subset=['账号'], keep='first', inplace=True)
renhang = renhang.sort_values(by=['账号']).reset_index(drop=True)

In [26]:
yanyin.sort_values(by=['开户日期'], ascending=False, inplace=True)
yanyin.drop_duplicates(subset=['账号'], keep='first', inplace=True)
yanyin = yanyin.sort_values(by=['账号']).reset_index(drop=True)

#### 账号异常值处理

In [27]:
hexin = hexin[hexin['账号'].map(lambda x: len(str(x))) >= 6].reset_index(drop=True)
renhang = renhang[renhang['账号'].map(lambda x: len(str(x))) >= 6].reset_index(drop=True)
yanyin = yanyin[yanyin['账号'].map(lambda x: len(str(x))) >= 6].reset_index(drop=True)

In [28]:
hexin.shape

(3083, 21)

In [29]:
renhang.shape

(3670, 22)

In [30]:
yanyin.shape

(2289, 25)

### 处理

#### 核心无人行有

In [224]:
hexin['账号'] = hexin['账号'].map(lambda x: str(x))
res1 = pd.DataFrame(list(set(renhang['账号']) - set(hexin['账号'])), columns=['人行账号'])
columns = ['人行账号', '账户名称', '账户性质', '开户日期', '状态']
res1 = res1.merge(renhang, how='inner', left_on='人行账号', right_on='账号')[columns]
res1.columns=['人行账号','人行账户名称','人行账户性质','人行开户日期','人行账户状态']
res1['备注']=None
res1.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res1.insert(0,column='机构号',value='419901')

#### 核心有人行无

In [223]:
res2 = pd.DataFrame(list(set(hexin['账号']) - set(renhang['账号'])), columns=['核心账号'])
columns=['核心账号','户名','账户类型','账户属性','开户日期','账户状态']
res2 = res2.merge(hexin, how='inner', left_on='核心账号', right_on='账号')[columns]
res2.columns=['核心账号','核心户名','核心账户类型','核心账户属性','核心开户日期','核心账户状态']
res2['备注']=None
res2.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res2.insert(0,column='机构号',value='419901')

#### 核心与人行户名不一致

In [225]:
res3=hexin.merge(renhang,how='inner',on='账号')
res3=res3[res3['户名']!=res3['账户名称']]
res3=res3[['账号','户名','账户名称','账户类型','账户属性','账户性质','开户日期_x','开户日期_y','账户状态','状态']]
res3.columns=['账号','核心户名','人行账户名称','核心账户类型','核心账户属性','人行账户性质','核心开户日期',
              '人行开户日期','核心账户状态','人行状态']
def pre3(x):
    x=x.replace('（','')
    x=x.replace('）','')
    x=x.replace('(','')
    x=x.replace(')','')
    x=x.replace('０','0')
    x=x.replace('１','1')
    x=x.replace('２','2')
    x=x.replace('３','3')
    x=x.replace('４','4')
    x=x.replace('５','5')
    x=x.replace('６','6')
    x=x.replace('７','7')
    x=x.replace('８','8')
    x=x.replace('９','9')
    return x
res3['核心户名_']=res3['核心户名'].map(lambda x: pre3(x))
res3['人行账户名称_']=res3['人行账户名称'].map(lambda x: pre3(x))
res3=res3[res3['核心户名_']!=res3['人行账户名称_']]
res3['备注']=None
res3.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res3.insert(0,column='机构号',value='419901')
res3.drop(columns=['核心户名_','人行账户名称_'],inplace=True)

#### 核心与人行账号属性不一致

In [221]:
res4=hexin.merge(renhang,how='inner',on='账号')
def pre4(x):
    if '基本' in x: return '基本'
    if '一般' in x: return '一般'
    if '专用' in x: return '专用'
    if '验资' in x: return '验资'
res4['账户属性_']=res4['账户属性'].apply(lambda x: pre4(x))
res4['账户性质_']=res4['账户性质'].apply(lambda x: pre4(x))
res4=res4[res4['账户属性_']!=res4['账户性质_']]
res4=res4[['账号','户名','账户名称','账户类型','账户属性','账户性质','开户日期_x','开户日期_y','账户状态','状态']]
res4.columns=['账号','核心户名','人行账户名称','核心账户类型','核心账户属性','人行账户性质','核心开户日期',
              '人行开户日期','核心账户状态','人行状态']
res4['备注']=None
res4.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res4.insert(0,column='机构号',value='419901')

#### 核心与人行账号状态不一致

In [222]:
res5=hexin.merge(renhang,how='inner',on='账号')
def pre5(x):
    if 'C-销户' in x: return '撤销'
    if 'D-不动户' in x: return '正常'
    if 'A-活动' in x: return '正常'
    if 'N-新建' in x: return '正常'
    if 'H-预开户' in x: return '正常'
    if 'S-久悬户' in x: return '久悬'
res5['账户状态_']=res5['账户状态'].apply(lambda x: pre5(x))
res5=res5[res5['账户状态_']!=res5['状态']]
res5=res5[['账号','户名','账户名称','账户类型','账户属性','账户性质','开户日期_x','开户日期_y','账户状态','状态']]
res5.columns=['账号','核心户名','人行账户名称','核心账户类型','核心账户属性','人行账户性质','核心开户日期',
              '人行开户日期','核心账户状态','人行状态']
res5['备注']=None
res5.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res5.insert(0,column='机构号',value='419901')

#### 核心有验印无

In [202]:
res6 = pd.DataFrame(list(set(hexin['账号']) - set(yanyin['账号'])),columns=['账号'])
columns = ['账号','户名','账户类型','账户属性','开户日期','账户状态']
res6 = res6.merge(hexin, how='inner', on='账号')[columns]
res6.columns=['核心账号','核心户名','核心账户类型','核心账户属性','核心开户日期','核心账户状态']
res6['备注']=None
res6.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res6.insert(0,column='机构号',value='419901')

#### 核心无验印有

In [206]:
res7 = pd.DataFrame(list(set(yanyin['账号']) - set(hexin['账号'])),columns=['账号'])
columns = ['账号','账户名称','账户属性','开户日期']
res7 = res7.merge(yanyin, how='inner', on='账号')[columns]
res7.columns=['验印账号','验印账户名称','验印账户属性','验印开户日期']
res7['备注']=None
res7.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res7.insert(0,column='机构号',value='419901')

#### 核心与验印账户属性不一致

In [219]:
res8=hexin.merge(yanyin,how='inner',on='账号')
res8['账户属性_x_']=res8['账户属性_x'].apply(lambda x: pre4(x))
res8['账户性质_y_']=res8['账户属性_y'].apply(lambda x: pre4(x))
res8=res8[res8['账户属性_x_']!=res8['账户性质_y_']]
res8=res8[['账号','户名','账户名称','账户类型','账户属性_x','账户属性_y','开户日期_x','开户日期_y','账户状态']]
res8.columns=['账号','核心户名','验印账户名称','核心账户类型','核心账户属性','验印账户属性','核心开户日期','验印开户日期','核心状态']
res8['备注']=None
res8.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res8.insert(0,column='机构号',value='419901')

#### 核心与验印户名不一致

In [228]:
res9=hexin.merge(yanyin,how='inner',on='账号')
res9=res9[res9['户名']!=res9['账户名称']]
res9=res9[['账号','户名','账户名称','账户类型','账户属性_x','账户属性_y','开户日期_x','开户日期_y','账户状态']]
res9.columns=['账号','核心户名','验印账户名称','核心账户类型','核心账户属性','验印账户属性','核心开户日期','验印开户日期','核心状态']
res9['核心户名_']=res9['核心户名'].map(lambda x: pre3(x))
res9['验印账户名称_']=res9['验印账户名称'].map(lambda x: pre3(x))
res9=res9[res9['核心户名_']!=res9['验印账户名称_']]
res9['备注']=None
res9.insert(0,column='机构名称',value='中原银行郑州商务外环路支行')
res9.insert(0,column='机构号',value='419901')
res9.drop(columns=['核心户名_','验印账户名称_'],inplace=True)

### 导出

In [232]:
writer=pd.ExcelWriter('分析结果.xlsx')

In [233]:
res1.to_excel(writer,'核心无人行有',index=False)
res2.to_excel(writer,'核心有人行无',index=False)
res3.to_excel(writer,'核心与人行户名不一致',index=False)
res4.to_excel(writer,'核心与人行账号属性不一致',index=False)
res5.to_excel(writer,'核心与人行账号状态不一致',index=False)
res6.to_excel(writer,'核心有验印无',index=False)
res7.to_excel(writer,'核心无验印有',index=False)
res8.to_excel(writer,'核心与验印账户属性不一致',index=False)
res9.to_excel(writer,'核心与验印户名不一致',index=False)

In [234]:
writer.save()

In [236]:
res9.dtypes

机构号       object
机构名称      object
账号        object
核心户名      object
验印账户名称    object
核心账户类型    object
核心账户属性    object
验印账户属性    object
核心开户日期     int64
验印开户日期    object
核心状态      object
备注        object
dtype: object